# Retail food store data

In [1]:
import json
import pandas as pd

# load the JSON file as a dictionary
with open('retail_food_stores.json', 'r', encoding='utf-8') as file:
    data = json.load(file)

# extract the metadata and store it in a dataframe
metadata = pd.DataFrame(data['meta'], index=[0])

metadata['temp'] = 1
# extract the data and store it in a dataframe
df = pd.DataFrame(data['data'])

df['temp'] = 1
# merge the metadata dataframe with the data dataframe
result = pd.merge(metadata, df, on='temp')

# display the resulting dataframe
print(result)

       view  temp                   0                                     1  \
0       NaN     1  row-656w.xr9g-izkp  00000000-0000-0000-830A-FB544A42623C   
1       NaN     1  row-sip5~swjf.mic4  00000000-0000-0000-1D72-DEFD8206793E   
2       NaN     1  row-h2ap-c7pr_kkgm  00000000-0000-0000-87EC-56223CC693F4   
3       NaN     1  row-ut8p_acga_u6xi  00000000-0000-0000-2B14-8A8858FE407B   
4       NaN     1  row-6ytw~jem2.4fnp  00000000-0000-0000-BDB9-7D5D5C295FDF   
...     ...   ...                 ...                                   ...   
28515   NaN     1  row-v9cd-gbc6.6u3b  00000000-0000-0000-698E-B98F201A4CB5   
28516   NaN     1  row-eizm.b2mt~yi3g  00000000-0000-0000-9CD9-EFBC6C615B05   
28517   NaN     1  row-ditr-fpqg-nayr  00000000-0000-0000-E01E-5F5CFE3BBE69   
28518   NaN     1  row-i766-7r6j-tuki  00000000-0000-0000-30C0-8088F99C64A1   
28519   NaN     1  row-uuzr~e7ps~79md  00000000-0000-0000-BD04-EC6A3A14BF2F   

       2           3     4           5     6    7  

In [2]:
column_names = list(result.columns.values)

print(column_names)

['view', 'temp', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [3]:
# Rename columns
result.rename(columns={
    'view': 'view',
    'temp': 'temp',
    0: 'sid',
    1: 'id',
    2: 'position',
    3: 'created_at',
    4: 'created_meta',
    5:'updated_at',
    6:'updated_meta',
    7:'meta_data',
    8:'County',
}, inplace=True)

column_names = list(result.columns.values)

print(column_names)

['view', 'temp', 'sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at', 'updated_meta', 'meta_data', 'County', 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


In [4]:
# Rename columns
result.rename(columns={
    9:'License_Number',
    10:'Operation_Type',
    11:'Establishment_Type',
    12:'Entity_Name',
    13:'DBA_Name',
    14:'Street_Number',
    15:'Street_Name',
    16:'Address_Line_2',
    17:'Address_Line_3',
    18:'City',
    19:'State',
    20:'Zip_Code',
    21:'Square_Footage',
    22:'Georeference',
    23:'NYS_Municipal_Boundaries'
}, inplace=True)

column_names = list(result.columns.values)

print(column_names)

['view', 'temp', 'sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at', 'updated_meta', 'meta_data', 'County', 'License_Number', 'Operation_Type', 'Establishment_Type', 'Entity_Name', 'DBA_Name', 'Street_Number', 'Street_Name', 'Address_Line_2', 'Address_Line_3', 'City', 'State', 'Zip_Code', 'Square_Footage', 'Georeference', 'NYS_Municipal_Boundaries']


# Create cassandra session

In [5]:
from cassandra.cluster import Cluster
from cassandra.policies import DCAwareRoundRobinPolicy
from cassandra.auth import PlainTextAuthProvider

#uth_provider = PlainTextAuthProvider(username='cassandra', password='password')
#cluster = Cluster(['127.0.0.1'], load_balancing_policy=DCAwareRoundRobinPolicy(local_dc='US-WEST'), port=9042, auth_provider=auth_provider)
auth_provider = PlainTextAuthProvider(username='Test', password='Test@4321')
# Connect to the Cassandra cluster
cluster = Cluster(['127.0.0.1'], port=9042, auth_provider=auth_provider)
session = cluster.connect('dapdb')


In [6]:
delete_stmnt = session.prepare('DROP TABLE IF EXISTS dapdb.retail_food_stores')
session.execute(delete_stmnt)

# Create table of Retail_Food_Stores

In [7]:
create_statement = session.prepare('''CREATE TABLE Retail_Food_Stores (id varchar PRIMARY KEY,County varchar,License_Number varchar,Operation_Type varchar,Establishment_Type varchar,Entity_Name varchar,DBA_Name varchar,Street_Number varchar,Street_Name varchar,Address_Line_2 varchar,Address_Line_3 varchar,City varchar,State varchar,Zip_Code varchar,Square_Footage varchar,Georeference varchar,NYS_Municipal_Boundaries varchar)''')

In [8]:
session.execute(create_statement)


In [9]:
insert_statement = session.prepare('''INSERT INTO retail_food_stores (id,County, License_Number, Operation_Type, Establishment_Type, Entity_Name, DBA_Name, Street_Number, Street_Name, Address_Line_2, Address_Line_3, City, State, Zip_Code, Square_Footage, Georeference, NYS_Municipal_Boundaries) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''')

In [10]:
for _, row in result.iterrows():
    session.execute(insert_statement, [row['id'],row['County'],row['License_Number'],row['Operation_Type'],row['Establishment_Type'],row['Entity_Name'],row['DBA_Name'],row['Street_Number'],row['Street_Name'],row['Address_Line_2'],row['Address_Line_3'],row['City'],row['State'],row['Zip_Code'],row['Square_Footage'],row['Georeference'],row['NYS_Municipal_Boundaries']])

# Farmers_Markets_in_New_York_State.json

In [11]:
import json
import pandas as pd

# load the JSON file as a dictionary
with open('Farmers_Markets_in_New_York_State.json', 'r', encoding='utf-8') as file:
    farmer_data = json.load(file)

# extract the metadata and store it in a dataframe
farmer_metadata = pd.DataFrame(farmer_data['meta'], index=[0])

farmer_metadata['temp'] = 1
# extract the data and store it in a dataframe
df_farmer = pd.DataFrame(farmer_data['data'])

df_farmer['temp'] = 1
# merge the metadata dataframe with the data dataframe
farmer_result = pd.merge(farmer_metadata, df_farmer, on='temp')

# display the resulting dataframe
print(farmer_result.head())

   view  temp                   0                                     1  2  \
0   NaN     1  row-8bvy~av76~b29y  00000000-0000-0000-28E0-E06AD96EE2A7  0   
1   NaN     1  row-s6nz~k9hb_cy4t  00000000-0000-0000-A626-382A53B00DBE  0   
2   NaN     1  row-3rrp_2m92-gba2  00000000-0000-0000-3EC6-E88B89C60758  0   
3   NaN     1  row-tvpw.ird8-ib3y  00000000-0000-0000-CF90-47907C6B0A6A  0   
4   NaN     1  row-aeje_yd3e.62nv  00000000-0000-0000-F62C-A22FDBBC23B5  0   

            3     4           5     6    7  ...  \
0  1680644578  None  1680644578  None  { }  ...   
1  1680644578  None  1680644578  None  { }  ...   
2  1680644578  None  1680644578  None  { }  ...   
3  1680644578  None  1680644578  None  { }  ...   
4  1680644578  None  1680644578  None  { }  ...   

                                            18                   19     20 21  \
0  Mon-Fri 10am-5pm, Sat 10a-4p  Sun 10a-4:30p   June 1-December 23      M  Y   
1                                  Sat 9am-1pm    May 6-Decemb

In [12]:
farmer_result_column_names = list(farmer_result.columns.values)

print(farmer_result_column_names)

['view', 'temp', 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]


In [13]:
# Rename columns
farmer_result.rename(columns={
    'view': 'view',
    'temp': 'temp',
    0: 'sid',
    1: 'id',
    2: 'position',
    3: 'created_at',
    4: 'created_meta',
    5:'updated_at',
    6:'updated_meta',
    7:'meta_data',
    8:'County',
}, inplace=True)

farmer_result_column_names = list(farmer_result.columns.values)

print(farmer_result_column_names)

['view', 'temp', 'sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at', 'updated_meta', 'meta_data', 'County', 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]


In [14]:
# Rename columns
farmer_result.rename(columns={
    9:'Market_Name',
    10:'Market_Location',
    11:'Address_Line_1',
    12:'City',
    13:'State',
    14:'Zip',
    15:'Contact',
    16:'Phone',
    17:'Market_Link',
    18:'Operation_Hours',
    19:'Operation_Season',
    20:'Operating_Months',
    21:'FMNP',
    22:'SNAP',
    23:'FCC_Issued',
    24:'FCC_Accepted',
    25:'Latitude',
    26:'Longitude',
    27:'Georeference_1'
}, inplace=True)

farmer_result_column_names = list(farmer_result.columns.values)

print(farmer_result_column_names)

['view', 'temp', 'sid', 'id', 'position', 'created_at', 'created_meta', 'updated_at', 'updated_meta', 'meta_data', 'County', 'Market_Name', 'Market_Location', 'Address_Line_1', 'City', 'State', 'Zip', 'Contact', 'Phone', 'Market_Link', 'Operation_Hours', 'Operation_Season', 'Operating_Months', 'FMNP', 'SNAP', 'FCC_Issued', 'FCC_Accepted', 'Latitude', 'Longitude', 'Georeference_1']


In [15]:
delete_stmnt = session.prepare('DROP TABLE IF EXISTS dapdb.Farmer_Market_NYC')
session.execute(delete_stmnt)

In [16]:
create_statement = session.prepare('''CREATE TABLE Farmer_Market_NYC (id varchar PRIMARY KEY,County varchar,Market_Name varchar,Market_Location varchar,Address_Line_1 varchar,City varchar,State varchar,Zip varchar,Contact varchar,Phone varchar,Operation_Hours varchar,Operation_Season varchar,Operating_Months varchar,FMNP varchar,SNAP varchar,FCC_Issued varchar, FCC_Accepted varchar,Latitude varchar,Longitude varchar,Georeference_1 varchar)''')

In [17]:
session.execute(create_statement)

In [18]:
insert_statement = session.prepare('''INSERT INTO Farmer_Market_NYC (id,County, Market_Name, Market_Location, Address_Line_1, City, State, Zip, Contact, Phone, Operation_Hours, Operation_Season, Operating_Months, FMNP, SNAP, FCC_Issued, FCC_Accepted, Latitude, Longitude, Georeference_1) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)''')
farmer_result.count()

view                  0
temp                405
sid                 405
id                  405
position            405
created_at          405
created_meta          0
updated_at          405
updated_meta          0
meta_data           405
County              405
Market_Name         405
Market_Location     405
Address_Line_1      405
City                405
State               405
Zip                 405
Contact             405
Phone               403
Market_Link         308
Operation_Hours     405
Operation_Season    405
Operating_Months    405
FMNP                405
SNAP                405
FCC_Issued          405
FCC_Accepted        405
Latitude            405
Longitude           405
Georeference_1      405
dtype: int64

In [19]:
for _, row in farmer_result.iterrows():
    session.execute(insert_statement, [row['id'],row['County'],row['Market_Name'],row['Market_Location'],row['Address_Line_1'],row['City'],row['State'],row['Zip'],row['Contact'],row['Phone'],row['Operation_Hours'],row['Operation_Season'],row['Operating_Months'],row['FMNP'],row['SNAP'],row['FCC_Issued'],row['FCC_Accepted'],row['Latitude'],row['Longitude'],row['Georeference_1']])